In [1]:
from elasticsearch import Elasticsearch
import yaml
import json
from collections import defaultdict

In [2]:
def get_config():
    with open('fastapi.yml', encoding='utf-8') as reader:
        conf = yaml.safe_load(reader)
    
    es_conf = conf.get("ELASTICSEARCH")

    return es_conf

In [3]:
conf = get_config()
es = Elasticsearch(**conf['SESSION_KWARGS'])
my_index = 'fbi-2022'
my_query = dict()
with open('my_query.json') as f:
    my_query = json.load(f)

In [4]:
response = es.search(index=my_index, body=my_query)
buckets = response['aggregations']['dirs']['buckets']

/home/users/adr8688/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  """Entry point for launching an IPython kernel.


In [7]:
def splitting_by_semicolon(my_dict):
    return {k: v.split(';') if ';' in v else v for k, v in my_dict.items()}

In [8]:
def agg_string_to_dict(agg_string):
    result = f'{{{agg_string}}}'
    result = result.replace('\t', '')
    result = result.replace('\\', '/')
    result = result.replace('\n', '')
    
    semicolons = [n for n, i in enumerate(result) if i ==';']
    for x, s in enumerate(semicolons):
        threshold = s - x * 2
        result = result[:threshold - 1] + ';' + result[threshold + 2:]
        
    result = json.loads(result)
    result = splitting_by_semicolon(result)

    return result

In [9]:
output = []
for b in buckets:
    try:
        output.append(agg_string_to_dict(b['key']))
    except Exception as e:
        print(b['key'])
        print(e)
    

In [12]:
with open('vars.json', 'w') as f:
    json.dump(output, f, indent=2)